In [ ]:
#import torch 
import pandas as pd 
import seaborn as sns 
import pandas as pd 

In [ ]:
import matplotlib.pyplot as plt 
import json
import re 

In [ ]:
property_data = pd.read_csv(r'datasets/datasets/property_data_set.csv') 
property_interactions = pd.read_csv(r'datasets/datasets/property_interactions.csv') 
property_photos = pd.read_csv(r'datasets/datasets/property_photos.tsv', sep = "\t")

NameError: name 'pd' is not defined

In [ ]:
property_data.shape

(28888, 21)

In [ ]:
property_data.columns

Index(['property_id', 'type', 'activation_date', 'bathroom', 'floor',
       'total_floor', 'furnishing', 'gym', 'latitude', 'longitude',
       'lease_type', 'lift', 'locality', 'parking', 'property_age',
       'property_size', 'swimming_pool', 'pin_code', 'rent', 'deposit',
       'building_type'],
      dtype='object')

In [ ]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 10)

In [ ]:
property_data.head()

,property_id,type,activation_date,bathroom,floor,total_floor,furnishing,gym,latitude,longitude,lease_type,lift,locality,parking,property_age,property_size,swimming_pool,pin_code,rent,deposit,building_type
0,ff808081469fd6e20146a5af948000ea,BHK2,09-03-2017 14:36,1,3,4.0,SEMI_FURNISHED,1,12.876174,77.596571,FAMILY,1,Hulimavu,BOTH,2,850,1,560076.0,12000,120000,AP
1,ff8080814702d3d10147068359d200cd,BHK2,07-03-2017 12:02,2,4,11.0,SEMI_FURNISHED,1,13.018444,77.678122,FAMILY,1,Ramamurthy Nagar,BOTH,1,1233,1,560016.0,20000,150000,AP
2,ff808081470c645401470fb03f5800a6,BHK2,10-03-2017 13:43,2,0,4.0,NOT_FURNISHED,1,12.975072,77.665865,ANYONE,1,GM Palya,FOUR_WHEELER,0,1200,0,560075.0,15000,75000,AP
3,ff808081470c6454014715eaa5960281,BHK3,09-03-2017 22:16,2,3,4.0,SEMI_FURNISHED,0,12.888169,77.591282,ANYONE,0,Arakere,BOTH,1,1300,0,560076.0,17000,150000,AP
4,ff808081474aa867014771a0298f0aa6,BHK1,15-03-2017 18:29,1,1,2.0,SEMI_FURNISHED,0,12.990243,77.712962,ANYONE,0,Hoodi,BOTH,4,450,0,560048.0,6500,40000,IF


In [ ]:
property_interactions

,property_id,request_date
0,ff808081469fd6e20146a5af948000ea,2017-03-10 17:42:34
1,ff808081469fd6e20146a5af948000ea,2017-03-09 15:51:17
2,ff808081469fd6e20146a5af948000ea,2017-03-10 17:30:22
3,ff808081469fd6e20146a5af948000ea,2017-03-11 17:48:46
4,ff8080814702d3d10147068359d200cd,2017-03-30 19:59:15
...,...,...
170606,ff8081815b2007fc015b201c77a20395,2017-04-03 16:13:55
170607,ff8081815b2007fc015b201c77a20395,2017-04-02 21:54:14
170608,ff8081815b2007fc015b201c77a20395,2017-04-09 11:33:14
170609,ff8081815b2007fc015b201c77a20395,2017-04-04 10:01:12


In [ ]:
def fix_photo_element(photo_element):
    if not photo_element: 
        return 0
    photo_element = photo_element.replace("\\",'') 
    photo_element = re.sub("title",r'"title',photo_element,count=1).strip("\"")
    return len(json.loads(photo_element))

In [ ]:
property_photos['photo_urls'] = property_photos['photo_urls'].fillna(value='')
property_photos['photo_count'] = property_photos['photo_urls'].apply(fix_photo_element)

In [ ]:
property_photos = property_photos.groupby('property_id')['photo_count'].sum().reset_index()

In [ ]:
property_data['activation_date'] = pd.to_datetime(property_data['activation_date'], format = '%d-%m-%Y %H:%M')
property_interactions['request_date'] = pd.to_datetime(property_interactions['request_date'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
num_req = pd.merge(
    property_data[["property_id", "activation_date"]],
    property_interactions,
    on="property_id",
    how="left",
)

In [ ]:
num_req['request_day'] = num_req['request_date'] - num_req['activation_date']

num_req['request_day'] = num_req['request_day'].dt.total_seconds() / (60 * 60 * 24)

In [ ]:
num_req['within_three_flag'] = num_req['request_day'] <= 3 
num_req['within_seven_flag'] = num_req['request_day'] <= 7 

In [ ]:
request_day_within_3d = (
    num_req.groupby("property_id")
    .agg({"within_three_flag": "sum"})
    .reset_index()
    .rename(columns={"within_three_flag": "request_day_within_3d"})
)


request_day_within_7d = (
    num_req.groupby("property_id")
    .agg({"within_three_flag": "sum"})
    .reset_index()
    .rename(columns={"within_three_flag": "request_day_within_7d"})
)

In [ ]:
request_day_within_3d

,property_id,request_day_within_3d
0,ff808081469fd6e20146a5af948000ea,4
1,ff8080814702d3d10147068359d200cd,3
2,ff808081470c645401470fb03f5800a6,15
3,ff808081470c6454014715eaa5960281,5
4,ff808081474aa867014771a0298f0aa6,25
...,...,...
28883,ff8081815b1fdda8015b1fe675ae0146,5
28884,ff8081815b1fdda8015b1ffee8a305d2,2
28885,ff8081815b1fdda8015b2000369205fd,0
28886,ff8081815b200540015b20158b0103c9,2


In [ ]:
request_day_within_7d

,property_id,request_day_within_7d
0,ff808081469fd6e20146a5af948000ea,4
1,ff8080814702d3d10147068359d200cd,3
2,ff808081470c645401470fb03f5800a6,15
3,ff808081470c6454014715eaa5960281,5
4,ff808081474aa867014771a0298f0aa6,25
...,...,...
28883,ff8081815b1fdda8015b1fe675ae0146,5
28884,ff8081815b1fdda8015b1ffee8a305d2,2
28885,ff8081815b1fdda8015b2000369205fd,0
28886,ff8081815b200540015b20158b0103c9,2


In [ ]:
def create_request_bucket(value):
    if value > 1 and value <= 2: 
        return '1_to_2' 
    elif value > 2 and value <= 5: 
        return '3_to_5' 
    elif value >= 5: 
        return "above_5" 
    else: 
        return "1_or_less"


In [ ]:
request_day_within_3d['3d_class'] = request_day_within_3d['request_day_within_3d'].apply(create_request_bucket) 
request_day_within_7d['7d_class'] = request_day_within_7d['request_day_within_7d'].apply(create_request_bucket) 
